# 1.加载必要的库

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# 2.定义超参数

In [2]:
BATCH_SIZE = 64  # 每批处理的数据
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 是使用GPU还是CPU训练
EPOCHS = 10  # 训练数据集的轮次

# 3.构建pipeline，对图像做处理

In [3]:
pipeline = transforms.Compose([
                                transforms.ToTensor(),  # 将图片转为tensor
                                transforms.Normalize((0.1307,), (0.3081,))  # 正则化：降低模型复杂度
                                ])

# 4.下载、加载数据

In [4]:
from torch.utils.data import DataLoader

# 下载数据集
train_set = datasets.MNIST("data", train=True, download=True, transform=pipeline)
test_set = datasets.MNIST("data", train=False, download=True, transform=pipeline)

# 加载数据
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
# 查看图片
with open("data/MNIST/raw/train-images-idx3-ubyte", "rb") as f:
    file = f.read()

image1 = [int(str(item).encode("ascii"), 16) for item in file[16: 16+784]]
print(image1)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 24, 24, 24, 294, 310, 373, 38, 358, 597, 583, 295, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 48, 54, 148, 340, 368, 595, 595, 595, 595, 595, 549, 370, 595, 578, 405, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 73, 568, 595, 595, 595, 595, 595, 595, 595, 595, 593, 147, 130, 130, 86, 57, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 537, 595, 595, 595, 595, 595, 408, 386, 583, 577, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 342, 263, 595, 595, 517, 17, 0, 67, 340, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 1, 340, 595, 1

In [6]:
import cv2
import numpy as np

image1_np = np.array(image1, dtype=np.uint8).reshape(28, 28, 1)
print(image1_np.shape)
cv2.imwrite("digit.jpg", image1_np)

(28, 28, 1)


True

# 5.构建网络模型

In [7]:
class Digit(nn.Module):
    def __init__(self):
        super(Digit, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)  # 1:灰度图片的通道，10:输出通道，5:kernel
        self.conv2 = nn.Conv2d(10, 20, 3)
        self.fc1 = nn.Linear(20*10*10, 500)  # 20*10*10:输入通道， 500：输出通道
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        input_size = x.size(0)  # batch_size*1*28*28
        x = self.conv1(x)  # batch_size*1*28*28 --> batch_size*10*24*24
        x = F.relu(x)
        x = F.max_pool2d(x, 2, 2)  # batch_size*10*24*24 --> batch_size*10*12*12

        x = self.conv2(x)  # batch_size*10*12*12 --> batch_size*20*10*10
        x = F.relu(x)
        x = x.view(input_size, -1)  # batch_size*20*10*10 --> batch_size*2000
        x = self.fc1(x)  # batch_size*2000 --> batch_size*500
        x = F.relu(x)  # 保持shape不变
        x = self.fc2(x)  # batch_size*500 --> batch_size*10

        output = F.log_softmax(x, dim=1)  # 计算分类后，每个数字的概率值

        return output

# 6.定义优化器

In [8]:
model = Digit().to(DEVICE)

optimizer = optim.Adam(model.parameters())

# 7.定义训练方法

In [9]:
def train_model(model, device, train_loader, optimizer, epoch):
    # 模型训练
    model.train()
    for batch_index, (data, target) in enumerate(train_loader):
        # 部署到DEVICE上去
        data, target = data.to(device), target.to(device)
        # 梯度初始化为0
        optimizer.zero_grad()
        # 训练后的结果
        output = model(data)
        # 计算损失
        loss = F.cross_entropy(output, target)
        # 找到概率值最大的下标
        pred = output.max(1, keepdim=True)  # pred = output.argmax(dim=1)
        # 反向传播
        loss.backward()
        # 参数更新
        optimizer.step()

        if batch_index % 3000 == 0:
            print("Train Epoch: {} \t Loss: {:.6f}".format(epoch, loss.item()))

# 8.定义测试方法

In [12]:
def test_model(model, device, test_loader):
    # 模型验证
    model.eval()
    # 正确率
    correct = 0.0
    # 测试损失
    test_loss = 0.0
    with torch.no_grad():  # 不会计算梯度，也不会进行反向传播
        for data, target in test_loader:
            # 部署到DEVICE上去
            data, target = data.to(device), target.to(device)
            # 测试数据
            output = model(data)
            # 计算测试损失
            test_loss += F.cross_entropy(output, target).item()
            # 找到概率值最大的下标
            pred = output.max(1, keepdim=True)[1]  # pred = output.argmax(dim=1)
            # 累计正确率
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print("TEST -- Average loss: {:.4f}, Accuracy: {:.3f}\n".format(test_loss, 100.0*correct/len(test_loader.dataset)))

# 9.调用方法7/8

In [13]:
for epoch in range(1, EPOCHS+1):
    train_model(model, DEVICE, train_loader, optimizer, epoch)
    test_model(model, DEVICE, test_loader)

Train Epoch: 1 	 Loss: 0.071365
TEST -- Average loss: 0.0005, Accuracy: 99.040

Train Epoch: 2 	 Loss: 0.010103
TEST -- Average loss: 0.0006, Accuracy: 98.780

Train Epoch: 3 	 Loss: 0.003717
TEST -- Average loss: 0.0004, Accuracy: 99.100

Train Epoch: 4 	 Loss: 0.021769
TEST -- Average loss: 0.0006, Accuracy: 98.830

Train Epoch: 5 	 Loss: 0.069066
TEST -- Average loss: 0.0006, Accuracy: 98.820

Train Epoch: 6 	 Loss: 0.000318
TEST -- Average loss: 0.0005, Accuracy: 99.100

Train Epoch: 7 	 Loss: 0.000029
TEST -- Average loss: 0.0005, Accuracy: 99.040

Train Epoch: 8 	 Loss: 0.001771
TEST -- Average loss: 0.0007, Accuracy: 98.950

Train Epoch: 9 	 Loss: 0.013838
TEST -- Average loss: 0.0006, Accuracy: 99.100

Train Epoch: 10 	 Loss: 0.000015
TEST -- Average loss: 0.0007, Accuracy: 99.140

